<a href="https://colab.research.google.com/github/ManuelBagasina/DATCapstone/blob/ML-Manuel/TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

In [63]:
!git clone -b ML-Manuel https://github.com/ManuelBagasina/DATCapstone.git
%cd DATCapstone/data

Cloning into 'DATCapstone'...
remote: Enumerating objects: 628, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 628 (delta 72), reused 25 (delta 25), pack-reused 517 (from 3)
Receiving objects: 100% (628/628), 66.93 MiB | 23.02 MiB/s, done.
Resolving deltas: 100% (325/325), done.
/content/DATCapstone/data/DATCapstone/data/DATCapstone/data/DATCapstone/data


In [64]:
from zipfile import ZipFile
import pandas as pd

# Explicitly extract data.csv from ZIP
with ZipFile('_data.csv.zip', 'r') as z:
    with z.open('data.csv') as f:  # Ignore macOS metadata files
        df = pd.read_csv(f, index_col=0)

# Convert date and sort
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by=['Ticker', 'Date'])


In [65]:
#Read Data
# df = pd.read_csv('/Users/wonminkim/Projects/DATCapstone/data2/data.csv', index_col=0)
# Correct path for ZIP file
zip_path = '/content/DATCapstone/data/_data.csv.zip'

# Extract specific file from ZIP
with ZipFile(zip_path, 'r') as z:
    with z.open('data.csv') as f:  # Explicitly specify the file to extract
        df = pd.read_csv(f, index_col=0)


df['Date'] = pd.to_datetime(df['Date'])

#Forward fill -> micro, macro, stock / fill na -> sentiment
df[df.columns[1:40]] = df.groupby('Ticker')[df.columns[1:40]].ffill()
df[df.columns[40:-5]] = df[df.columns[40:-5]].fillna(0)

df = df.dropna(subset=df.columns[0:-5])

# Sorting
df = df.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

# Lag feature (Exclude embedding columns)
lag_cols = df.columns[1:44]
lags = [1, 3, 5, 7, 14, 30]
lag_features = []

for col in lag_cols:
    for lag in lags:
        lagged = df.groupby('Ticker')[col].shift(lag)
        lag_features.append(lagged.rename(f'{col}_lag{lag}'))

lag_df = pd.concat(lag_features, axis=1)

df = pd.concat([df.reset_index(drop=True), lag_df.reset_index(drop=True)], axis=1)

In [66]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,GDP (Billions USD),Unemployment Rate (%),...,neutral_lag5,neutral_lag7,neutral_lag14,neutral_lag30,positive_lag1,positive_lag3,positive_lag5,positive_lag7,positive_lag14,positive_lag30
0,2021-09-30,140.983670,141.690258,138.648004,138.863907,89056700,0.0,0.0,23921.991,4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-01,139.256450,140.257452,136.518433,139.992477,94639600,0.0,0.0,24777.038,4.5,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,2021-10-04,139.119037,139.560665,135.694064,136.547852,98322000,0.0,0.0,24777.038,4.5,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
3,2021-10-05,136.891371,139.590139,136.763788,138.481186,80861100,0.0,0.0,24777.038,4.5,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
4,2021-10-06,136.871718,139.501783,135.792205,139.354584,83221100,0.0,0.0,24777.038,4.5,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,2025-01-27,124.788521,128.388181,116.689260,118.409103,818830900,0.0,0.0,29723.864,4.0,...,0.571282,0.000000,0.999946,0.001172,0.000712,0.053954,0.048698,0.000000,0.000014,0.000014
5628,2025-01-28,121.798801,128.988143,116.239315,128.978149,579666400,0.0,0.0,29723.864,4.0,...,0.083897,0.333974,0.256342,0.007833,0.182257,0.013230,0.374254,0.000178,0.199403,0.000346
5629,2025-01-29,126.488367,126.878331,120.038963,123.688622,467120600,0.0,0.0,29723.864,4.0,...,0.140121,0.571282,0.181732,0.032715,0.119110,0.000712,0.053954,0.048698,0.013047,0.000541
5630,2025-01-30,123.088674,124.988501,118.089134,124.638535,392925500,0.0,0.0,29723.864,4.0,...,0.307131,0.083897,0.476841,0.000166,0.001830,0.182257,0.013230,0.374254,0.153314,0.001441


In [67]:
import glob
import pandas as pd

# Correct path for article CSVs in Google Colab
csv_files = glob.glob('/content/DATCapstone/data/*articles*.csv')
print("Found files:", csv_files)  # Debugging


Found files: ['/content/DATCapstone/data/AVGO_articles..csv', '/content/DATCapstone/data/META_articles..csv', '/content/DATCapstone/data/TSLA_articles..csv', '/content/DATCapstone/data/ADBE_articles.csv', '/content/DATCapstone/data/NVDA_articles..csv', '/content/DATCapstone/data/MSFT_articles..csv', '/content/DATCapstone/data/AAPL_articles.csv', '/content/DATCapstone/data/NFLX_articles..csv', '/content/DATCapstone/data/GOOGL_articles..csv', '/content/DATCapstone/data/AMZN_articles..csv']


In [68]:
!ls /content/DATCapstone/data/*articles*.csv


/content/DATCapstone/data/AAPL_articles.csv
/content/DATCapstone/data/ADBE_articles.csv
/content/DATCapstone/data/AMZN_articles..csv
/content/DATCapstone/data/AVGO_articles..csv
/content/DATCapstone/data/GOOGL_articles..csv
/content/DATCapstone/data/META_articles..csv
/content/DATCapstone/data/MSFT_articles..csv
/content/DATCapstone/data/NFLX_articles..csv
/content/DATCapstone/data/NVDA_articles..csv
/content/DATCapstone/data/TSLA_articles..csv


In [69]:
if not csv_files:
    print("No article files found. Initializing empty columns.")
    df_articles = pd.DataFrame(columns=['Date', 'Ticker', 'Articles_Negative', 'Articles_Neutral', 'Articles_positive'])
else:
    # Your existing processing code
    df_articles_list = []
    for file in csv_files:
        ticker = os.path.basename(file).split('_')[0]
        df_articles = pd.read_csv(file)
        df_articles['Ticker'] = ticker
        df_articles_list.append(df_articles)

    df_articles = pd.concat(df_articles_list, ignore_index=True)

    # Preprocessing steps...
# Articles dataframe preprocessing
df_articles["Date"] = pd.to_datetime(df_articles["time"], errors='coerce', utc=True).dt.tz_convert(None).dt.date
df_articles = df_articles[['Date', 'Ticker', 'sentiment']]
sentiment_dummies = pd.get_dummies(df_articles['sentiment'])
df_encoded = pd.concat([df_articles[['Date', 'Ticker']], sentiment_dummies], axis=1)
df_articles = df_encoded.groupby(['Date', 'Ticker']).sum().reset_index()
df_articles['Date'] = pd.to_datetime(df_articles['Date'])

df_articles['articles_avg_sentiment'] = (df_articles['Negative'] * -1 + df_articles['Neutral'] * 0 + df_articles['Positive']) / (df_articles['Negative'] + df_articles['Neutral'] + df_articles['Positive'])
df_articles.columns = ['Date', 'Ticker', 'Articles_Negative', 'Articles_Neutral', 'Articles_positive', 'Articles_avg_sentiment']
df_articles

df = pd.merge(df, df_articles, on=['Date', 'Ticker'], how = 'left')

# Fill 0 except the target columns
df[[col for col in df.columns if 'Target' not in col]] = df[[col for col in df.columns if 'Target' not in col]].fillna(0)

In [70]:
import glob

# Read articles df
csv_files = glob.glob(os.path.join("*_articles*.csv")) # Updated to reflect file naming pattern

df_articles_list = []

for file in csv_files:
    # get ticker from filename
    ticker = os.path.basename(file).split('_')[0]
    df_articles = pd.read_csv(file)
    df_articles['Ticker'] = ticker
    df_articles_list.append(df_articles)

# concat articles dataframe
df_articles = pd.concat(df_articles_list, ignore_index=True)

# Articles dataframe preprocessing
df_articles["Date"] = pd.to_datetime(df_articles["time"], errors='coerce', utc=True).dt.tz_convert(None).dt.date
df_articles = df_articles[['Date', 'Ticker', 'sentiment']]
sentiment_dummies = pd.get_dummies(df_articles['sentiment'])
df_encoded = pd.concat([df_articles[['Date', 'Ticker']], sentiment_dummies], axis=1)
df_articles = df_encoded.groupby(['Date', 'Ticker']).sum().reset_index()
df_articles['Date'] = pd.to_datetime(df_articles['Date'])

df_articles['articles_avg_sentiment'] = (df_articles['Negative'] * -1 + df_articles['Neutral'] * 0 + df_articles['Positive']) / (df_articles['Negative'] + df_articles['Neutral'] + df_articles['Positive'])
df_articles.columns = ['Date', 'Ticker', 'Articles_Negative', 'Articles_Neutral', 'Articles_positive', 'Articles_avg_sentiment']
df_articles

df = pd.merge(df, df_articles, on=['Date', 'Ticker'], how = 'left')

# Fill 0 except the target columns
df[[col for col in df.columns if 'Target' not in col]] = df[[col for col in df.columns if 'Target' not in col]].fillna(0)

In [71]:
print(df_articles.columns)

Index(['Date', 'Ticker', 'Articles_Negative', 'Articles_Neutral',
       'Articles_positive', 'Articles_avg_sentiment'],
      dtype='object')


In [72]:
from sklearn.decomposition import PCA

# PCA
pca = PCA(n_components=0.95, random_state=42)
emb_cols = [col for col in df.columns if col.startswith("emb_")]
X_emb = df[emb_cols]
X_emb_pca = pca.fit_transform(X_emb)

# PCA columns
pca_columns = [f"emb_pca_{i}" for i in range(X_emb_pca.shape[1])]
df_pca_part = pd.DataFrame(X_emb_pca, columns=pca_columns, index=df.index)
df_non_pca = df.drop(columns=emb_cols)

df_pca = pd.concat([df_non_pca.reset_index(drop=True), df_pca_part.reset_index(drop=True)], axis=1)

# 결과 확인
df_pca.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,GDP (Billions USD),Unemployment Rate (%),...,emb_pca_74,emb_pca_75,emb_pca_76,emb_pca_77,emb_pca_78,emb_pca_79,emb_pca_80,emb_pca_81,emb_pca_82,emb_pca_83
0,2021-09-30,140.983670,141.690258,138.648004,138.863907,89056700,0.0,0.0,23921.991,4.7,...,-0.000573,-0.000315,-0.000716,-0.000052,0.000158,0.000176,0.000412,-0.000354,-0.000744,0.000022
1,2021-10-01,139.256450,140.257452,136.518433,139.992477,94639600,0.0,0.0,24777.038,4.5,...,-0.000573,-0.000315,-0.000716,-0.000052,0.000158,0.000176,0.000412,-0.000354,-0.000744,0.000022
2,2021-10-04,139.119037,139.560665,135.694064,136.547852,98322000,0.0,0.0,24777.038,4.5,...,-0.000573,-0.000315,-0.000716,-0.000052,0.000158,0.000176,0.000412,-0.000354,-0.000744,0.000022
3,2021-10-05,136.891371,139.590139,136.763788,138.481186,80861100,0.0,0.0,24777.038,4.5,...,-0.000573,-0.000315,-0.000716,-0.000052,0.000158,0.000176,0.000412,-0.000354,-0.000744,0.000022
4,2021-10-06,136.871718,139.501783,135.792205,139.354584,83221100,0.0,0.0,24777.038,4.5,...,-0.000573,-0.000315,-0.000716,-0.000052,0.000158,0.000176,0.000412,-0.000354,-0.000744,0.000022


# TFT Model 1

## Libraries

In [73]:
# Install
!pip install pytorch-lightning
!pip install pytorch-forecasting

In [74]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import torch

In [75]:
# PyTorch Lightning and PyTorch Forecasting
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss, MAE, RMSE
from pytorch_forecasting.data import GroupNormalizer, NaNLabelEncoder

# Scikit-learn for metrics and preprocessing
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA

## Setup

In [76]:
# Set random seeds for reproducibility
pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [77]:
# ========== 2. Resolving the NaN/Inf Target Issue ==========
# First, inspect Target_1day before any operations
print(f"Original dataframe shape: {df.shape}")
print(f"Total NaN in Target_1day: {df['Target_1day'].isna().sum()}")
print(f"Total inf in Target_1day: {np.isinf(df['Target_1day']).sum()}")

# Create a deep copy to avoid modifying the original df
working_df = df.copy()

# Very aggressive approach to handle Target_1day
# 1. First, let's print some statistics about Target_1day
target_val = working_df['Target_1day']
print(f"\nTarget_1day stats before cleaning:")
print(f"Min: {target_val.min()}")
print(f"Max: {target_val.max()}")
print(f"Mean: {target_val.mean()}")
print(f"Std: {target_val.std()}")

# 2. Replace NaN and inf with 0 in the target column
working_df['Target_1day'] = working_df['Target_1day'].replace([np.nan, np.inf, -np.inf], 0)

# 3. Verify the target is free of NaN and inf
print(f"\nAfter replacement:")
print(f"NaN in Target_1day: {working_df['Target_1day'].isna().sum()}")
print(f"Inf in Target_1day: {np.isinf(working_df['Target_1day']).sum()}")

# ========== 3. Additional Feature Checks ==========
# Create time index for TFT
working_df = working_df.sort_values(['Ticker', 'Date'])
working_df['time_idx'] = working_df.groupby('Ticker').cumcount()

# Define simple feature sets - using only crucial features that don't have NaN issues
categorical_vars = ['Ticker']

# Focus on a minimal set of relevant financial features
basic_features = [
    'Dividends', 'Stock Splits',
    'GDP (Billions USD)', 'Unemployment Rate (%)',
    'Producer Price Index (PPI)', 'Consumer Confidence Index',
    'Total Revenue', 'Operating Income', 'Net Income',
    'EBITDA', 'Gross Profit'
]

# Check which features exist and don't have NaN issues
valid_features = []
for col in basic_features:
    if col in working_df.columns:
        nan_count = working_df[col].isna().sum()
        inf_count = np.isinf(working_df[col]).sum()

        if nan_count > 0 or inf_count > 0:
            print(f"Feature {col}: {nan_count} NaNs, {inf_count} infs - filling with 0")
            working_df[col] = working_df[col].replace([np.nan, np.inf, -np.inf], 0)

        valid_features.append(col)

# Final features
continuous_vars = valid_features

print(f"\nSelected features:")
print(f"- Categorical ({len(categorical_vars)}): {categorical_vars}")
print(f"- Continuous ({len(continuous_vars)}): {continuous_vars}")
print(f"Total features: {len(categorical_vars) + len(continuous_vars)}")


Original dataframe shape: (5632, 1083)
Total NaN in Target_1day: 7
Total inf in Target_1day: 0

Target_1day stats before cleaning:
Min: -42.44155883789063
Max: 43.88923645019531
Mean: 0.10996016370985243
Std: 3.587426056358569

After replacement:
NaN in Target_1day: 0
Inf in Target_1day: 0

Selected features:
- Categorical (1): ['Ticker']
- Continuous (11): ['Dividends', 'Stock Splits', 'GDP (Billions USD)', 'Unemployment Rate (%)', 'Producer Price Index (PPI)', 'Consumer Confidence Index', 'Total Revenue', 'Operating Income', 'Net Income', 'EBITDA', 'Gross Profit']
Total features: 12


In [78]:
# ========== 4. Create Training and Validation Sets ==========
# Set prediction parameters
MAX_PREDICTION_LENGTH = 1  # Predict 1 day ahead
MAX_ENCODER_LENGTH = 30    # Use 30 days of history for prediction

# Split data into train and validation sets
cutoff_date = working_df['Date'].max() - timedelta(days=30)
train_df = working_df[working_df['Date'] <= cutoff_date].copy()
val_df = working_df[working_df['Date'] > cutoff_date].copy()

print(f"\nTraining data: {train_df.shape} rows, {train_df['Date'].min()} to {train_df['Date'].max()}")
print(f"Validation data: {val_df.shape} rows, {val_df['Date'].min()} to {val_df['Date'].max()}")

# Final verification - ensure there are absolutely no NaN or inf values in any used column
TARGET = "Target_1week"

print("\nFinal verification:")
for col in [TARGET] + continuous_vars:
    nan_count = train_df[col].isna().sum()
    inf_count = np.isinf(train_df[col]).sum()
    if nan_count > 0 or inf_count > 0:
        print(f"Warning: {col} still has {nan_count} NaNs and {inf_count} infs - fixing")
        train_df[col] = train_df[col].replace([np.nan, np.inf, -np.inf], 0)

    nan_count = val_df[col].isna().sum()
    inf_count = np.isinf(val_df[col]).sum()
    if nan_count > 0 or inf_count > 0:
        print(f"Warning: {col} still has {nan_count} NaNs and {inf_count} infs in validation - fixing")
        val_df[col] = val_df[col].replace([np.nan, np.inf, -np.inf], 0)


Training data: (5492, 1084) rows, 2021-06-30 00:00:00 to 2024-12-31 00:00:00
Validation data: (140, 1084) rows, 2025-01-02 00:00:00 to 2025-01-31 00:00:00

Final verification:


In [79]:
# ========== 5. Create TFT Datasets ==========
try:
    print("\nCreating TimeSeriesDataSet...")

    # Create an additional indicator column for where we filled values
    train_df['filled_indicator'] = 0.0
    val_df['filled_indicator'] = 0.0

    train_dataset = TimeSeriesDataSet(
        data=train_df,
        time_idx="time_idx",
        target=TARGET,
        group_ids=["Ticker"],
        max_encoder_length=MAX_ENCODER_LENGTH,
        max_prediction_length=MAX_PREDICTION_LENGTH,
        static_categoricals=categorical_vars,
        # Include the filled indicator with our features
        time_varying_known_reals=continuous_vars + ['filled_indicator'],
        time_varying_unknown_reals=[TARGET],
        target_normalizer=GroupNormalizer(
            groups=["Ticker"],
            transformation="softplus"  # Handles positive and negative values
        ),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True
    )

    # Create validation dataset from training dataset
    val_dataset = TimeSeriesDataSet.from_dataset(
        train_dataset, val_df, predict=True, stop_randomization=True
    )

    # Create dataloaders
    batch_size = 64
    train_dataloader = train_dataset.to_dataloader(
        train=True, batch_size=batch_size, num_workers=0, shuffle=True
    )
    val_dataloader = val_dataset.to_dataloader(
        train=False, batch_size=batch_size, num_workers=0, shuffle=False
    )

    print("TimeSeriesDataSet created successfully!")

except Exception as e:
    print(f"Error creating TimeSeriesDataSet: {str(e)}")
    raise


Creating TimeSeriesDataSet...
Error creating TimeSeriesDataSet: 2481 (45.17%) of Target_1week values were found to be NA or infinite (even after encoding). NA values are not allowed `allow_missing_timesteps` refers to missing rows, not to missing values. Possible strategies to fix the issue are (a) dropping the variable Target_1week, (b) using `NaNLabelEncoder(add_nan=True)` for categorical variables, (c) filling missing values and/or (d) optionally adding a variable indicating filled values


ValueError: 2481 (45.17%) of Target_1week values were found to be NA or infinite (even after encoding). NA values are not allowed `allow_missing_timesteps` refers to missing rows, not to missing values. Possible strategies to fix the issue are (a) dropping the variable Target_1week, (b) using `NaNLabelEncoder(add_nan=True)` for categorical variables, (c) filling missing values and/or (d) optionally adding a variable indicating filled values

In [ ]:
# ========== 6. Create and Train TFT Model ==========
# Create the TFT model
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.001,
    hidden_size=32,       # Reduced size to avoid overfitting
    attention_head_size=2,
    dropout=0.2,          # Increased dropout for regularization
    hidden_continuous_size=16,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=3,
    weight_decay=1e-2,
)

print(f"TFT model size: {tft.size()/1e3:.1f}k parameters")

# Define callbacks
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=5,
    verbose=True,
    mode="min"
)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=15,        # Reduced epochs for faster training
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, lr_monitor],
    enable_model_summary=True,
    limit_train_batches=50,  # For faster training (remove for full training)
    limit_val_batches=20,    # For faster training (remove for full training)
)

print("\nStarting TFT model training...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)
print("Training completed!")

# Save the model
model_path = "tft_stock_model.ckpt"
trainer.save_checkpoint(model_path)
print(f"Model saved to {model_path}")

In [ ]:
# ========== 7. Evaluate Model ==========
# Make predictions
predictions = tft.predict(
    val_dataloader,
    return_x=True,
    return_y=True,
    mode="prediction"
)

# Extract actual and predicted values
actuals = []
predicted = []
tickers = []

# Get validation data
for x, y in iter(val_dataloader):
    actuals.extend(y[0].cpu().numpy().flatten())
    batch_tickers = x["groups"]["Ticker"]
    tickers.extend(batch_tickers)

# Get median predictions (quantile=0.5)
predicted = predictions.output.prediction.cpu().numpy()[:, 0, 0]

# Ensure we have the same number of predictions as actuals
min_len = min(len(actuals), len(predicted), len(tickers))
actuals = actuals[:min_len]
predicted = predicted[:min_len]
tickers = tickers[:min_len]

# Create results dataframe
results_df = pd.DataFrame({
    "Ticker": tickers,
    "Actual": actuals,
    "Predicted": predicted,
    "Error": predicted - actuals
})

# Calculate metrics
mae = mean_absolute_error(actuals, predicted)
mse = mean_squared_error(actuals, predicted)
rmse = np.sqrt(mse)
r2 = r2_score(actuals, predicted)

print("\n----- Validation Metrics -----")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# ========== 8. Visualize Results ==========
# Plot overall results
plt.figure(figsize=(12, 6))
sample_size = min(100, len(actuals))
plt.plot(actuals[:sample_size], label='Actual')
plt.plot(predicted[:sample_size], label='Predicted')
plt.title(f'Actual vs Predicted {TARGET}')
plt.xlabel('Sample Index')
plt.ylabel('Price Change')
plt.legend()
plt.grid(True)
plt.savefig('tft_prediction_results.png')
plt.show()

# Plot results by ticker
plt.figure(figsize=(15, 10))
ticker_list = list(results_df["Ticker"].unique())[:4]  # First 4 tickers
for i, ticker in enumerate(ticker_list):
    ticker_df = results_df[results_df["Ticker"] == ticker]
    if len(ticker_df) > 0:
        plt.subplot(2, 2, i+1)
        plt.plot(ticker_df["Actual"].values, label=f'Actual ({ticker})')
        plt.plot(ticker_df["Predicted"].values, label=f'Predicted ({ticker})')
        plt.title(f'Ticker: {ticker}')
        plt.xlabel('Sample Index')
        plt.ylabel('Price Change')
        plt.legend()
        plt.grid(True)
plt.tight_layout()
plt.savefig('tft_ticker_predictions.png')
plt.show()

# Analyze feature importance
print("\nAnalyzing feature importance...")
interpretation = tft.interpret_output(predictions.x, predictions.output)

# Plot feature importance
plt.figure(figsize=(15, 10))
tft.plot_interpretation(interpretation)
plt.tight_layout()
plt.savefig('tft_feature_importance.png')
plt.show()

# Print top features
if "encoder_variables" in interpretation:
    encoder_vars = tft.encoder_variables
    encoder_importance = interpretation["encoder_variables"][0].mean(0).mean(0).cpu().numpy()

    print("\nTop most important features:")
    for i, (var, imp) in enumerate(sorted(zip(encoder_vars, encoder_importance),
                                      key=lambda x: x[1], reverse=True)[:10]):
        print(f"{i+1}. {var}: {imp:.4f}")

print("\nTFT model evaluation completed!")